In [13]:
# Program places orders

# STATUS: Incomplete

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=1) # kavi TWS live
ib=IB().connect('127.0.0.1', 4001, clientId=1) # kavi IBG live

In [14]:
import pandas as pd
import datetime

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
base_rom = 20    # 90% RoM atleast

df_orders = pd.read_pickle(datapath+'df_orders.pkl')

# Filter for puts below strike threshold
df_orders = df_orders[df_orders.Strike <= df_orders.Threshold]

# Filter out the top Return on Margin trades
df_orders1 = df_orders[df_orders.RoM >= base_rom].reset_index(drop=True)
df_orders1 = df_orders1.sort_values('RoM', ascending=False)

# Compute margin for the trade
df_orders1['Margin'] = df_orders1.marginpct*df_orders1.Mlot*df_orders1.Strike

In [15]:
# Filter only the trades expiring in this month
# df = df_orders1[df_orders1.DTE == df_orders1.DTE.min()].reset_index(drop=True)
df = df_orders1

In [ ]:
# sorted(df_orders1.ibSymbol.unique())
df.Expiry.min()

In [ ]:
# total margin
df.Margin.sum()

In [16]:
len(df)

11

In [ ]:
# Sort by Symbol for easy reference in Excel
df.sort_values(by='ibSymbol').to_excel(datapath+'temp.xlsx', index=None, freeze_panes=(1,1))

In [ ]:
df.ibSymbol.unique()

In [17]:
#------------------------------------------------
#####    WARNING: THIS PLACES THE TRADE  #######
#________________________________________________

limitTrade = [ib.placeOrder(contract, order) 
              for contract, order in zip(df.Contract, df.Orders)]

Error 388, reqId 68: Order size 8,500 is smaller than the minimum required size of 13,000.
Canceled order: Trade(contract=Option(conId=339359530, symbol='TV18BRDCS', lastTradeDateOrContractMonth='20190131', strike=28.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='TV18BRDCST19JAN28PE', tradingClass='TV18BRDCST'), order=LimitOrder(softDollarTier=2505990946264: Name: , Value: , DisplayName: , orderId=68, clientId=1, action='SELL', totalQuantity=8500, lmtPrice=4.25), orderStatus=OrderStatus(status='Cancelled'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2018, 12, 18, 1, 34, 40, 892448, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2018, 12, 18, 1, 34, 40, 903475, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 388, reqId 68: Order size 8,500 is smaller than the minimum required size of 13,000.')])
Error 388, reqId 69: Order size 7,000 is smaller than the minimum required size o

#------------------------------------------------
#####         Code to cancel all trades    ############
#________________________________________________
cancel = ib.reqGlobalCancel()


In [ ]:
limitTrade

In [20]:
# Put placed orders to a dataframe
df_orders_placed = pd.DataFrame(limitTrade)

In [21]:
# Write orders placed to file
x  =  datapath + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_orders_placed.xlsx"
writer = pd.ExcelWriter(x)
df_orders_placed.to_excel(writer, 'all-options', index=False, header=None)
writer.save()

In [18]:
ib.disconnect()

In [ ]:
list(df_orders)

In [ ]:
df.head()